In [1]:
import detectron2
import torch
from detectron2.utils.logger import setup_logger, log_every_n_seconds
setup_logger()
import os, json, cv2, random
from detectron2 import model_zoo
from detectron2.engine import DefaultTrainer, DefaultPredictor, HookBase
from detectron2.config import get_cfg
from detectron2.utils import comm
from detectron2.utils.visualizer import Visualizer, ColorMode
from detectron2.data import MetadataCatalog, DatasetCatalog
from detectron2.data import DatasetMapper, build_detection_test_loader
from detectron2.data.datasets.coco import convert_to_coco_json
from detectron2.evaluation import COCOEvaluator
from detectron2.structures import BoxMode
import logging
import glob
import re
import numpy as np
import matplotlib.pyplot as plt
import pickle
import math
import time
import datetime

In [2]:
cat={'rooftops':0} #Order should be same in MetadataCatalog
def get_dict(img_dir):
    os.chdir(img_dir)
    count = 1
    file_list=[]
    for file in glob.glob("*.json"):
        file_list.append(file)
    dataset_dicts = []
    for file in file_list:
        json_file = os.path.join(img_dir, file)
        with open(json_file) as f:
            imgs = json.load(f)
            
   
        record = {}
        filename = os.path.join(img_dir,imgs['imagePath'])
        height,width = imgs['imageHeight'],imgs['imageWidth']
        record["file_name"] = filename
        record["image_id"] = count
        record["height"] = height
        record["width"] = width
        objs = []
        for i in range(len(imgs['shapes'])):
            tmp = imgs['shapes'][i]['points']
            tmp_mat = np.matrix(tmp)
            xmin,xmax,ymin,ymax = np.min(tmp_mat[:,0]),np.max(tmp_mat[:,0]),np.min(tmp_mat[:,1]),np.max(tmp_mat[:,1])
            poly = list(np.ravel(tmp_mat))
            obj = {
                        "bbox": [xmin, ymin, xmax, ymax],
                        "bbox_mode": BoxMode.XYXY_ABS,
                        "segmentation": [poly],
                        "category_id": cat[imgs['shapes'][i]['label']],
                    }
            objs.append(obj)
        record["annotations"] = objs
        dataset_dicts.append(record)
        count += 1
    return dataset_dicts

In [10]:
im_dir = # <directory with a folder named "train" containing labelled images>
for d in ['train']:
    DatasetCatalog.register("<datasetname>_train" ,lambda d=d:get_dict(im_dir+'/' +d))
    MetadataCatalog.get('<datasetname>_train').set(thing_classes=['rooftops'])
project_metadata = MetadataCatalog.get("<datasetname>_train")

In [8]:
cfg = get_cfg()
cfg.merge_from_file(model_zoo.get_config_file("COCO-InstanceSegmentation/mask_rcnn_R_101_FPN_3x.yaml")) # Pre-Trained model architecture
cfg.DATASETS.TRAIN = ("<datasetname>_train",) # Custom Dataset
cfg.DATASETS.TEST = ()
cfg.DATALOADER.NUM_WORKERS = 8 #Define number of workers according to the machine
cfg.MODEL.WEIGHTS = model_zoo.get_checkpoint_url("COCO-InstanceSegmentation/mask_rcnn_R_101_FPN_3x.yaml") #Pre-Trained model weights
cfg.SOLVER.IMS_PER_BATCH = 1 # Number of images per batch
cfg.SOLVER.BASE_LR = 0.0025 #Base learning rate {Tweak as per dataser size and epochs}
cfg.SOLVER.MAX_ITER =50000 #Maximum Epochs
cfg.SOLVER.STEPS = (5000,12000,20000,35000) #Decreasing the learning rate after given epochs
cfg.MODEL.ROI_HEADS.BATCH_SIZE_PER_IMAGE = 128 
cfg.MODEL.ROI_HEADS.NUM_CLASSES = 1 # Define number of classes in the dataset
cfg.OUTPUT_DIR = os.path.join("<'path in which output weights will be saved'>")

In [ ]:
# Run as it is
os.makedirs(cfg.OUTPUT_DIR, exist_ok=True)
trainer = DefaultTrainer(cfg) 
trainer.resume_or_load(resume=False) # resume True if using trained weights
trainer.train()